In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [10]:
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

In [11]:
# AndhraPradesh bus route

In [12]:
 def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name
 def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)# Close the driver
df.to_csv('andrhra_bus_details.csv', index=False)
driver.quit()
            

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/madanapalli-to-bangalore: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF616489632+30946]
	(No symbol) [0x00007FF61643E3C9]
	(No symbol) [0x00007FF616336FDA]
	(No symbol) [0x00007FF61638822C]
	(No symbol) [0x00007FF61638850C]
	(No symbol) [0x00007FF6163CDCB7]
	(No symbol) [0x00007FF6163ACAAF]
	(No symbol) [0x00007FF6163CB041]
	(No symbol) [0x00007FF6163AC813]
	(No symbol) [0x00007FF61637A6E5]
	(No symbol) [0x00007FF61637B021]
	GetHandleVerifier [0x00007FF6165BF83D+1301229]
	GetHandleVerifier [0x00007FF6165CBDB7+1351783]
	GetHandleVerifier [0x00007FF6165C2A03+1313971]
	GetHandleVerifier [0x00007FF6164BDD06+245686]
	(No symbol) [0x00007FF61644758F]
	(No symbol) [0x00007FF616443804]
	(No symbol) [0x00007FF616443992]
	(No symbol) [0x00007FF61643A3EF]
	BaseThreadInitThunk [0x00007FFF46E8257D+29]
	RtlUserThreadStart [0x00007FFF47CAAF28+40]



In [13]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 40331,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:40,04h 50m,18:30,4.0,INR 396,23 Seats available
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 1600,RAJDHANI (A.C. Semi Sleeper),14:00,05h 40m,19:40,2.7,INR 480,27 Seats available
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 40362,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",14:20,06h 10m,20:30,2.9,INR 396,20 Seats available
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 1432,RAJDHANI (A.C. Semi Sleeper),14:20,04h 55m,19:15,2.2,INR 480,32 Seats available
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 1402,RAJDHANI (A.C. Semi Sleeper),14:40,04h 50m,19:30,2.1,INR 480,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
2160,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,IRA Travels,A/C Sleeper (2+1),23:55,06h 15m,06:10,4.4,INR 999,13 Seats available
2161,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,NS Holidays,Bharat Benz A/C Seater /Sleeper (2+1),18:55,06h 20m,01:15,4.0,INR 2749,16 Seats available
2162,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,Yes Express Tours and Travels,Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1),23:30,05h 20m,04:50,3.4,INR 1000,20 Seats available
2163,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,Yes Express Tours and Travels,Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1),23:35,06h 45m,06:20,3.4,INR 3000,33 Seats available
